In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import simfin as sf
#Used for column names
from simfin.names import *

#SIMFIN params
# Where all the csvs are stored
sf.set_data_dir('./simfin_data/')
# Simfin needs an api key (Using simfins free key so only get stock info from before 2018 )
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')
# offset the fundamental data by 50 days the simfin documentation also
#recommends doing this..
dateOffset = pd.DateOffset(days=50)
refresh_days = 25
refresh_days_shareprices = 14

In [2]:
#Data collection from Simfin
hub = sf.StockHub(market='us', offset=dateOffset,refresh_days=refresh_days,refresh_days_shareprices=refresh_days_shareprices)
#Creating a panda dataframe with the simfin data
growthSignalsDf = hub.growth_signals(variant='daily')
valueSignalsDf = hub.val_signals(variant='daily')
financialSignalsDf = hub.fin_signals(variant='daily')
#Combining the 3 data frames into one big data frame
dfs = [financialSignalsDf, growthSignalsDf, valueSignalsDf]
signalsDf = pd.concat(dfs, axis=1)

Dataset "us-income-ttm" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-income-quarterly" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (1 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (1 days old).
- Loading from disk ... Done!
Cache-file 'growth_signals-d4ce5d48.pickle' on disk (1 days old).
- Loading from disk ... Done!
Cache-file 'val_signals-65cf454d.pickle' on disk (1 days old).
- Loading from disk ... Done!
Cache-file 'fin_signals-d4ce5d48.pickle' on disk (1 days old).
- Loading from disk ... Done!


In [3]:
#Drop the rows where all elements are missing.
signalsDf.dropna(how='all').head() 
df = signalsDf.dropna(how='all').reset_index(drop=True)

#Columns must have atleast 80% non NULL values, any that don't are dropped
#(Scikit cannot work well with lots of missing data)
thresh = 0.80 * len(signalsDf.dropna(how='all'))
signalsDf = signalsDf.dropna(axis='columns', thresh=thresh)
signalsDf.dropna(how='all').head()

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2010-09-20                           0.171492        0.553297   
       2010-09-21                           0.171492        0.553297   
       2010-09-22                           0.171492        0.553297   
       2010-09-23                           0.171492        0.553297   
       2010-09-24                           0.171492        0.553297   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2010-09-20               0.622857       1.966061             -0.0   
       2010-09-21               0.622857       1.966061             -0.0   
       2010-09-22               0.622857       1.966061             -0.0   
       2010-09-23               0.622857       1.966061             -0.0   
       2010-09-24               0.622857       1.966061             -0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2010-09-20             0.542205           5.636364     9.701999   
       2010-09-21             0.542205           5.636364     9.701999   
       2010-09-22             0.542205           5.636364     9.701999   
       2010-09-23             0.542205           5.636364     9.701999   
       2010-09-24             0.542205           5.636364     9.701999   

                   Net Profit Margin  Quick Ratio  ...  Earnings Yield  \
Ticker Date                                        ...                   
A      2010-09-20            0.08282     1.065135  ...        0.038314   
       2010-09-21            0.08282     1.065135  ...        0.038166   
       2010-09-22            0.08282     1.065135  ...        0.038650   
       2010-09-23            0.08282     1.065135  ...        0.038903   
       2010-09-24            0.08282     1.065135  ...        0.037048   

                   FCF Yield    Market-Cap    P/Cash        P/E      P/FCF  \
Ticker Date                                                                  
A      2010-09-20   0.041254  1.088377e+10  4.697355  26.100174  24.240028   
       2010-09-21   0.041095  1.092586e+10  4.715521  26.201109  24.333769   
       2010-09-22   0.041616  1.078907e+10  4.656483  25.873070  24.029109   
       2010-09-23   0.041889  1.071892e+10  4.626206  25.704844  23.872873   
       2010-09-24   0.039891  1.125557e+10  4.857819  26.991769  25.068079   

                      P/NCAV  P/NetNet   P/Sales  Price to Book Value  
Ticker Date                                                            
A      2010-09-20 -19.610401 -3.584315  2.161623             3.873229  
       2010-09-21 -19.686239 -3.598176  2.169983             3.888207  
       2010-09-22 -19.439766 -3.553127  2.142814             3.839527  
       2010-09-23 -19.313369 -3.530025  2.128882             3.814562  
       2010-09-24 -20.280302 -3.706757  2.235465             4.005540  

[5 rows x 35 columns]

In [4]:
# Name of the new column for the returns.
#This is the column the AI will attempt to predict
TOTAL_RETURN_1_3Y = 'Total Log Return 1-3 Years'
# Calculate the mean log-returns for all 1-3 year periods.
df_returns_1_3y = hub.mean_log_returns(name=TOTAL_RETURN_1_3Y,future=True, annualized=True,min_years=1, max_years=3)
#combine the two dataframes together
dfs = [signalsDf, df_returns_1_3y]
df_sig_rets = pd.concat(dfs, axis=1)

Cache-file 'mean_log_change-cb2ba940.pickle' on disk (0 days old).
- Loading from disk ... Done!


In [5]:
# Remove data outliers by winsorizing both the original stock data and the the Total return column
df_sig_rets = sf.winsorize(df_sig_rets)
# Remove all rows with any missing values
df_sig_rets = df_sig_rets.dropna(how='any')
# Remove all Stocks which have less than 150 data-rows.
df_sig_rets = df_sig_rets.groupby(TICKER).filter(lambda df: len(df)>150)

In [6]:
df_sig_rets.head()

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2011-09-19                            0.18595        0.738375   
       2011-09-20                            0.18595        0.738375   
       2011-09-21                            0.18595        0.738375   
       2011-09-22                            0.18595        0.738375   
       2011-09-23                            0.18595        0.738375   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2011-09-19               0.603113       3.470432             -0.0   
       2011-09-20               0.603113       3.470432             -0.0   
       2011-09-21               0.603113       3.470432             -0.0   
       2011-09-22               0.603113       3.470432             -0.0   
       2011-09-23               0.603113       3.470432             -0.0   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2011-09-19              0.53257          13.927536     9.810434   
       2011-09-20              0.53257          13.927536     9.810434   
       2011-09-21              0.53257          13.927536     9.810434   
       2011-09-22              0.53257          13.927536     9.810434   
       2011-09-23              0.53257          13.927536     9.810434   

                   Net Profit Margin  Quick Ratio  ...  FCF Yield  \
Ticker Date                                        ...              
A      2011-09-19           0.157048     2.669767  ...   0.077832   
       2011-09-20           0.157048     2.669767  ...   0.077632   
       2011-09-21           0.157048     2.669767  ...   0.084372   
       2011-09-22           0.157048     2.669767  ...   0.086961   
       2011-09-23           0.157048     2.669767  ...   0.085891   

                     Market-Cap    P/Cash        P/E      P/FCF     P/NCAV  \
Ticker Date                                                                  
A      2011-09-19  1.243702e+10  4.010648  12.253222  12.848161  18.562716   
       2011-09-20  1.246904e+10  4.020973  12.284766  12.881237  18.610504   
       2011-09-21  1.147294e+10  3.699754  11.303387  11.852208  17.123787   
       2011-09-22  1.113142e+10  3.589622  10.966914  11.499398  16.614056   
       2011-09-23  1.127016e+10  3.634363  11.103606  11.642727  16.821134   

                    P/NetNet   P/Sales  Price to Book Value  \
Ticker Date                                                   
A      2011-09-19 -31.818776  1.924342             2.961195   
       2011-09-20 -31.818776  1.929296             2.968818   
       2011-09-21 -31.818776  1.775172             2.731652   
       2011-09-22 -31.818776  1.722330             2.650338   
       2011-09-23 -31.818776  1.743797             2.683371   

                   Total Log Return 1-3 Years  
Ticker Date                                    
A      2011-09-19                    0.164458  
       2011-09-20                    0.163450  
       2011-09-21                    0.209664  
       2011-09-22                    0.226803  
       2011-09-23                    0.220432  

[5 rows x 36 columns]

In [7]:
df_sig_rets.columns

Index(['(Dividends + Share Buyback) / FCF', 'Asset Turnover',
       'CapEx / (Depr + Amor)', 'Current Ratio', 'Dividends / FCF',
       'Gross Profit Margin', 'Interest Coverage', 'Log Revenue',
       'Net Profit Margin', 'Quick Ratio', 'Return on Assets',
       'Return on Equity', 'Share Buyback / FCF', 'Assets Growth',
       'Assets Growth QOQ', 'Assets Growth YOY', 'Earnings Growth',
       'Earnings Growth QOQ', 'Earnings Growth YOY', 'FCF Growth',
       'FCF Growth QOQ', 'FCF Growth YOY', 'Sales Growth', 'Sales Growth QOQ',
       'Sales Growth YOY', 'Earnings Yield', 'FCF Yield', 'Market-Cap',
       'P/Cash', 'P/E', 'P/FCF', 'P/NCAV', 'P/NetNet', 'P/Sales',
       'Price to Book Value', 'Total Log Return 1-3 Years'],
      dtype='object')

In [8]:
#Originally was saving to a csv but was having troubles keeping the dataframe original
#df_sig_rets.to_csv (r'/mnt/c/Users/danie/Documents/319 A2/simfin-tutorials/stockdata.csv', header=True)

In [9]:
#Serializes the data frame exactly as it is to storage
df_sig_rets.to_pickle("./stockdata.pkl")

In [13]:
!jupyter nbconvert --to script *.ipynb

[NbConvertApp] Converting notebook 10_Neural_Networks.ipynb to script
[NbConvertApp] Writing 31883 bytes to 10_Neural_Networks.py
[NbConvertApp] Converting notebook Collection and Cleansing.ipynb to script
[NbConvertApp] Writing 2992 bytes to Collection and Cleansing.py
[NbConvertApp] Converting notebook LoadAndTrainData.ipynb to script
[NbConvertApp] Writing 6490 bytes to LoadAndTrainData.py


In [12]:
print(len(df_sig_rets.columns))

36
